# 데이터 준비

In [2]:
import nltk

nltk.download("punkt")
from nltk.tokenize import word_tokenize

from torchtext.legacy.data import Field
from torchtext.legacy.data import TabularDataset
from torchtext.legacy.data import BucketIterator
from torchtext.legacy.data import Iterator

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
DATA_PATH = "/content/drive/Othercomputers/내 컴퓨터/Sat_english/data/processed"

필드 정의

In [4]:
# 문장 필드
TEXT = Field(
    sequential=True,    # 문장 입력
    use_vocab=True,     
    tokenize=word_tokenize,     # nltk의 word_tokenize로 트큰화
    lower=True,         # 모두 소문자 처리
    batch_first=True,
)

# 정답 필드
LABEL = Field(
    sequential=False,
    use_vocab=False,
    batch_first=True,
)

데이터 불러오기

In [5]:
# CoLA 데이터 = 사전 학습 데이터
cola_train_data, cola_valid_data, cola_test_data = TabularDataset.splits(
    path=DATA_PATH,
    train="cola_train.tsv",
    validation="cola_valid.tsv",
    test="cola_test.tsv",
    format="tsv",
    fields=[("text", TEXT), ("label", LABEL)],
    skip_header=1,          # column명이 있는 1열 생략
)

TEXT.build_vocab(cola_train_data, min_freq=2)   # CoLA 데이터로 사전학습할 단어장 생성(2번 이상 나온 단어만)

# 수능 데이터 = 추가 학습 데이터
sat_train_data, sat_valid_data, sat_test_data = TabularDataset.splits(
    path=DATA_PATH,
    train="sat_train.tsv",
    validation="sat_valid.tsv",
    test="sat_test.tsv",
    format="tsv",
    fields=[("text", TEXT), ("label", LABEL)],
    skip_header=1,
)

DataLoader 정의

In [6]:
# CoLA 데이터
cola_train_iterator, cola_valid_iterator, cola_test_iterator = BucketIterator.splits(
    (cola_train_data, cola_valid_data, cola_test_data),
    batch_size=32,
    device=None,
    sort=False,
)

# 수능 데이터
sat_train_iterator, sat_valid_iterator, sat_test_iterator = BucketIterator.splits(
    (sat_train_data, sat_valid_data, sat_test_data),
    batch_size=8,
    device=None,
    sort=False,
)